In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


# Liberies

In [2]:
# Standard Libraries
import numpy as np
import pandas as pd

# Machine Learning Models and Metrics
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Time Series Libraries
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA

# Machine Learning Frameworks
import xgboost as xgb
from catboost import CatBoostRegressor, Pool
from lightgbm import LGBMRegressor


# For Suppressing Warnings
import warnings
warnings.filterwarnings("ignore")

# 1. Data 

In [3]:
# Load the training and test datasets
train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')

In [4]:
# add the 'dataset' column to distinguish train and test data
train['dataset'] = 'train'
test['dataset'] = 'test'

# concatenate the datasets with the added 'dataset' column
df = pd.concat([train, test], axis=0).reset_index(drop=True)


In [5]:
oil_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
holidays_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
store_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
transactions_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')

In [6]:
print(df.columns)
print(oil_data.columns)
print(holidays_data.columns)
print(store_data.columns)
print(transactions_data.columns)

Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion', 'dataset'], dtype='object')
Index(['date', 'dcoilwtico'], dtype='object')
Index(['date', 'type', 'locale', 'locale_name', 'description', 'transferred'], dtype='object')
Index(['store_nbr', 'city', 'state', 'type', 'cluster'], dtype='object')
Index(['date', 'store_nbr', 'transactions'], dtype='object')


In [7]:
train.head()

,id,date,store_nbr,family,sales,onpromotion,dataset
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,train
1,1,2013-01-01,1,BABY CARE,0.0,0,train
2,2,2013-01-01,1,BEAUTY,0.0,0,train
3,3,2013-01-01,1,BEVERAGES,0.0,0,train
4,4,2013-01-01,1,BOOKS,0.0,0,train


In [8]:
test

,id,date,store_nbr,family,onpromotion,dataset
0,3000888,2017-08-16,1,AUTOMOTIVE,0,test
1,3000889,2017-08-16,1,BABY CARE,0,test
2,3000890,2017-08-16,1,BEAUTY,2,test
3,3000891,2017-08-16,1,BEVERAGES,20,test
4,3000892,2017-08-16,1,BOOKS,0,test
...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,test
28508,3029396,2017-08-31,9,PREPARED FOODS,0,test
28509,3029397,2017-08-31,9,PRODUCE,1,test
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,test


In [9]:
oil_data

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


In [10]:
holidays_data

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


In [11]:
holidays_data[holidays_data['transferred'] == True]

,date,type,locale,locale_name,description,transferred
19,2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
72,2013-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
135,2014-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
255,2016-05-24,Holiday,National,Ecuador,Batalla de Pichincha,True
266,2016-07-25,Holiday,Local,Guayaquil,Fundacion de Guayaquil,True
268,2016-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True
297,2017-01-01,Holiday,National,Ecuador,Primer dia del ano,True
303,2017-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,True
312,2017-05-24,Holiday,National,Ecuador,Batalla de Pichincha,True
324,2017-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True


In [12]:
holidays_data[holidays_data['type'] == 'Transfer']

,date,type,locale,locale_name,description,transferred
20,2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
73,2013-10-11,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
136,2014-10-10,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
256,2016-05-27,Transfer,National,Ecuador,Traslado Batalla de Pichincha,False
265,2016-07-24,Transfer,Local,Guayaquil,Traslado Fundacion de Guayaquil,False
269,2016-08-12,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False
298,2017-01-02,Transfer,National,Ecuador,Traslado Primer dia del ano,False
304,2017-04-13,Transfer,Local,Cuenca,Fundacion de Cuenca,False
313,2017-05-26,Transfer,National,Ecuador,Traslado Batalla de Pichincha,False
325,2017-08-11,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False


In [13]:
holidays_data[holidays_data['type'] == 'Bridge']

,date,type,locale,locale_name,description,transferred
35,2012-12-24,Bridge,National,Ecuador,Puente Navidad,False
39,2012-12-31,Bridge,National,Ecuador,Puente Primer dia del ano,False
156,2014-12-26,Bridge,National,Ecuador,Puente Navidad,False
160,2015-01-02,Bridge,National,Ecuador,Puente Primer dia del ano,False
277,2016-11-04,Bridge,National,Ecuador,Puente Dia de Difuntos,False


In [14]:
holidays_data[holidays_data['type'] == 'Transfer']

,date,type,locale,locale_name,description,transferred
20,2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
73,2013-10-11,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
136,2014-10-10,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
256,2016-05-27,Transfer,National,Ecuador,Traslado Batalla de Pichincha,False
265,2016-07-24,Transfer,Local,Guayaquil,Traslado Fundacion de Guayaquil,False
269,2016-08-12,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False
298,2017-01-02,Transfer,National,Ecuador,Traslado Primer dia del ano,False
304,2017-04-13,Transfer,Local,Cuenca,Fundacion de Cuenca,False
313,2017-05-26,Transfer,National,Ecuador,Traslado Batalla de Pichincha,False
325,2017-08-11,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False


In [15]:
holidays_data.loc[holidays_data['type'] == 'Transfer', 'type'] = 'Holiday'

In [16]:
holidays_data['Holiday summary'] = (
    holidays_data['type'].astype(str) + " " +
    holidays_data['locale'].astype(str) + " " +
    holidays_data['locale_name'].astype(str) + " " 
    # holidays_data['description'].astype(str)
)

In [17]:
holidays_data= holidays_data.drop(columns=['type', 'locale', 'locale_name', 'description'] , errors='ignore' )

In [18]:
holidays_data['Holiday summary'].value_counts()

Holiday summary
Holiday National Ecuador                            68
Event National Ecuador                              56
Additional National Ecuador                         40
Holiday Local Latacunga                             12
Holiday Local Riobamba                              12
Holiday Local Guaranda                              12
Holiday Local Ambato                                12
Holiday Local Cuenca                                 7
Holiday Local Quito                                  7
Holiday Local Ibarra                                 7
Holiday Local Puyo                                   6
Holiday Local Libertad                               6
Holiday Regional Cotopaxi                            6
Holiday Local Manta                                  6
Holiday Local Esmeraldas                             6
Holiday Local Cayambe                                6
Holiday Local El Carmen                              6
Holiday Local Santo Domingo                      

In [19]:
store_data

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4
5,6,Quito,Pichincha,D,13
6,7,Quito,Pichincha,D,8
7,8,Quito,Pichincha,D,8
8,9,Quito,Pichincha,B,6
9,10,Quito,Pichincha,C,15


In [20]:
transactions_data

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
...,...,...,...
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932


In [21]:
store_data['state'].value_counts()

state
Pichincha                         19
Guayas                            11
Azuay                              3
Santo Domingo de los Tsachilas     3
Manabi                             3
Los Rios                           2
Cotopaxi                           2
Tungurahua                         2
El Oro                             2
Bolivar                            1
Imbabura                           1
Chimborazo                         1
Pastaza                            1
Santa Elena                        1
Loja                               1
Esmeraldas                         1
Name: count, dtype: int64

In [22]:
store_data['city'].value_counts()

city
Quito            18
Guayaquil         8
Cuenca            3
Santo Domingo     3
Latacunga         2
Machala           2
Manta             2
Ambato            2
Cayambe           1
Riobamba          1
Ibarra            1
Salinas           1
Puyo              1
Guaranda          1
Quevedo           1
Babahoyo          1
Daule             1
Playas            1
Loja              1
Libertad          1
Esmeraldas        1
El Carmen         1
Name: count, dtype: int64

In [23]:
# Merge oil prices data on date
df = df.merge(oil_data, on='date', how='left')

In [24]:
# Merge holiday data on date
df = df.merge(holidays_data, on='date', how='left')

In [25]:
# Merge store data on store number
df = df.merge(store_data, on='store_nbr', how='left')

In [26]:
# Merge transaction data on store number and date
df = df.merge(transactions_data, on=['date', 'store_nbr'], how='left')

In [27]:
df.sample(30)

,id,date,store_nbr,family,sales,onpromotion,dataset,dcoilwtico,transferred,Holiday summary,city,state,type,cluster,transactions
2796542,2750210,2017-03-28,25,MAGAZINES,7.000,0,train,48.36,NaN,NaN,Salinas,Santa Elena,D,1,836.0
666729,657819,2014-01-06,16,PRODUCE,719.106,0,train,93.12,NaN,NaN,Santo Domingo,Santo Domingo de los Tsachilas,C,3,897.0
2828658,2781123,2017-04-14,42,HOME AND KITCHEN I,11.000,1,train,NaN,False,Holiday Local Libertad,Cuenca,Azuay,D,2,1104.0
278273,276491,2013-06-05,17,HOME APPLIANCES,0.000,0,train,93.66,NaN,NaN,Quito,Pichincha,C,12,1251.0
1452787,1433185,2015-03-18,21,POULTRY,0.000,0,train,44.63,NaN,NaN,Santo Domingo,Santo Domingo de los Tsachilas,B,6,NaN
550029,542901,2013-11-01,41,HOME CARE,0.000,0,train,94.56,NaN,NaN,Machala,El Oro,D,4,930.0
2607463,2562913,2016-12-12,20,BABY CARE,3.000,0,train,52.74,NaN,NaN,Quito,Pichincha,B,6,1333.0
1310563,1290961,2014-12-28,31,BABY CARE,0.000,0,train,NaN,NaN,NaN,Babahoyo,Los Rios,B,10,1707.0
1141541,1125503,2014-09-25,39,BREAD/BAKERY,525.000,1,train,93.59,NaN,NaN,Cuenca,Azuay,B,6,1190.0
1286874,1270836,2014-12-16,17,CELEBRATION,4.000,0,train,55.97,NaN,NaN,Quito,Pichincha,C,12,1499.0


## 1.1: Basic Data Overview

In [28]:
# Basic info and shape
print("Dataset Shape:", df.shape)
print("\nDataset Info:")
df.info()

# Checking for missing values
print("\nMissing Values in Each Column:")
print(df.isnull().sum())

Dataset Shape: (3082860, 15)

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3082860 entries, 0 to 3082859
Data columns (total 15 columns):
 #   Column           Dtype  
---  ------           -----  
 0   id               int64  
 1   date             object 
 2   store_nbr        int64  
 3   family           object 
 4   sales            float64
 5   onpromotion      int64  
 6   dataset          object 
 7   dcoilwtico       float64
 8   transferred      object 
 9   Holiday summary  object 
 10  city             object 
 11  state            object 
 12  type             object 
 13  cluster          int64  
 14  transactions     float64
dtypes: float64(3), int64(4), object(8)
memory usage: 352.8+ MB

Missing Values in Each Column:
id                       0
date                     0
store_nbr                0
family                   0
sales                28512
onpromotion              0
dataset                  0
dcoilwtico          962280
transferred        25

In [29]:
# Summary statistics for numerical columns
print("\nSummary Statistics for Numerical Columns:")
df.describe()


Summary Statistics for Numerical Columns:


,id,store_nbr,sales,onpromotion,dcoilwtico,cluster,transactions
count,3.082860e+06,3.082860e+06,3.054348e+06,3.082860e+06,2.120580e+06,3.082860e+06,2.805231e+06
mean,1.518250e+06,2.750000e+01,3.590209e+02,2.657692e+00,6.780640e+01,8.481481e+00,1.697071e+03
std,8.742912e+05,1.558579e+01,1.107286e+03,1.236626e+01,2.564571e+01,4.649735e+00,9.668317e+02
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,2.619000e+01,1.000000e+00,5.000000e+00
25%,7.618048e+05,1.400000e+01,0.000000e+00,0.000000e+00,4.646000e+01,4.000000e+00,1.046000e+03
50%,1.521828e+06,2.750000e+01,1.100000e+01,0.000000e+00,5.325000e+01,8.500000e+00,1.395000e+03
75%,2.272940e+06,4.100000e+01,1.960110e+02,0.000000e+00,9.572000e+01,1.300000e+01,2.081000e+03
max,3.029399e+06,5.400000e+01,1.247170e+05,7.410000e+02,1.106200e+02,1.700000e+01,8.359000e+03


# 2. Data Preprocessing

## 2.1: Handle Missing Values

In [30]:
# Handle missing values in the dataset

# Fill missing values in oil prices using forward fill, then backward fill
df['dcoilwtico'] = df['dcoilwtico'].fillna(method='ffill').fillna(method='bfill')

# Create a binary column 'is_holiday'
# Set 0 for 'Not Holiday' and 1 for any holiday-related value
df['is_holiday'] = df['Holiday summary'].fillna('Not Holiday').apply(lambda x: 0 if x == 'Not Holiday' else 1)

# Handle missing values in the 'transactions' column:
# Step 1: Fill missing transactions with the median transactions for each product family
df['transactions'] = df.groupby('family')['transactions'].transform(
    lambda x: x.fillna(x.median())
)

# Step 2: For any remaining missing values, apply forward fill
df['transactions'].fillna(method='ffill', inplace=True)

# Step 3: If forward fill fails (e.g., for the first row), fill with the overall mean
df['transactions'].fillna(df['transactions'].mean(), inplace=True)

# Drop unnecessary columns
df = df.drop(columns=['transferred', 'Holiday summary'], errors='ignore')

In [31]:
# Checking for missing values
print("Missing values in each column:")
print(df.isnull().sum())

Missing values in each column:
id                  0
date                0
store_nbr           0
family              0
sales           28512
onpromotion         0
dataset             0
dcoilwtico          0
city                0
state               0
type                0
cluster             0
transactions        0
is_holiday          0
dtype: int64


## 2.2 Convert Date Column and Extract Temporal Features

In [32]:
# Extract year, month, day, day of the week, and other possible temporal features
# Convert 'date' column to numerical features
df['date'] = pd.to_datetime(df['date'])
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

df = df.sort_values(by='date')

## 2.3: Encode Categorical Variables

In [33]:
df['family'].value_counts()
df.groupby('family', as_index=False).agg(
    sales_count=('sales', 'count'),
    sales_mean=('sales', 'mean')
).sort_values(by='sales_mean', ascending=False)[:10]

,family,sales_count,sales_mean
12,GROCERY I,92556,3790.432797
3,BEVERAGES,92556,2394.912701
30,PRODUCE,92556,1355.373698
7,CLEANING,92556,1074.171518
8,DAIRY,92556,711.175991
5,BREAD/BAKERY,92556,464.150612
28,POULTRY,92556,351.078816
24,MEATS,92556,341.965905
25,PERSONAL CARE,92556,271.192381
9,DELI,92556,265.629746


In [34]:
df.groupby('store_nbr', as_index=False).agg(
    sales_count=('sales', 'count'),
    sales_mean=('sales', 'mean')
).sort_values(by='sales_mean', ascending=False)[:10]

,store_nbr,sales_count,sales_mean
43,44,56562,1120.118405
44,45,56562,984.565998
46,47,56562,919.777871
2,3,56562,911.098054
48,49,56562,784.039156
45,46,56562,756.775349
47,48,56562,649.584599
50,51,56562,594.106667
7,8,56562,550.264615
49,50,56562,517.551554


In [35]:
df.columns

Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion', 'dataset',
       'dcoilwtico', 'city', 'state', 'type', 'cluster', 'transactions',
       'is_holiday', 'day', 'day_of_week', 'month', 'year'],
      dtype='object')

In [36]:
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder


# One-Hot Encoding for selected columns
# Perform one-hot encoding on 'store_type' and 'family' columns
df = pd.get_dummies(df, columns=['store_nbr', 'family'], dtype=int, drop_first=True)

# Function for Target Encoding multiple categorical columns
def target_encoding_multiple_columns(df, target_column, categorical_columns, dataset_column='dataset'):
    
    for col in categorical_columns:
        # Calculate target mean for each category in the column using training data
        target_mean = df[df[dataset_column] == 'train'].groupby(col)[target_column].mean()

        # Map the target mean encoding to the entire dataset
        df[f'{col}_encoded'] = df[col].map(target_mean)

        # Handle missing values for categories not present in training
        df[f'{col}_encoded'].fillna(df[target_column].mean(), inplace=True)

    return df

# Example usage: Apply Target Encoding to multiple columns
categorical_columns = ['month', 'day', 'year', 'day_of_week']
df = target_encoding_multiple_columns(df, target_column='sales', categorical_columns=categorical_columns)


# Drop unnecessary columns after encoding
columns_to_drop = [
    'locale_name', 'description', 'transferred', 'city', 'state', 'family',
    'month', 'day_of_week', 'day', 'year', 'cluster', 'type'
]
df.drop(columns=columns_to_drop, errors='ignore', inplace=True)

In [37]:
df.columns

Index(['id', 'date', 'sales', 'onpromotion', 'dataset', 'dcoilwtico',
       'transactions', 'is_holiday', 'store_nbr_2', 'store_nbr_3',
       'store_nbr_4', 'store_nbr_5', 'store_nbr_6', 'store_nbr_7',
       'store_nbr_8', 'store_nbr_9', 'store_nbr_10', 'store_nbr_11',
       'store_nbr_12', 'store_nbr_13', 'store_nbr_14', 'store_nbr_15',
       'store_nbr_16', 'store_nbr_17', 'store_nbr_18', 'store_nbr_19',
       'store_nbr_20', 'store_nbr_21', 'store_nbr_22', 'store_nbr_23',
       'store_nbr_24', 'store_nbr_25', 'store_nbr_26', 'store_nbr_27',
       'store_nbr_28', 'store_nbr_29', 'store_nbr_30', 'store_nbr_31',
       'store_nbr_32', 'store_nbr_33', 'store_nbr_34', 'store_nbr_35',
       'store_nbr_36', 'store_nbr_37', 'store_nbr_38', 'store_nbr_39',
       'store_nbr_40', 'store_nbr_41', 'store_nbr_42', 'store_nbr_43',
       'store_nbr_44', 'store_nbr_45', 'store_nbr_46', 'store_nbr_47',
       'store_nbr_48', 'store_nbr_49', 'store_nbr_50', 'store_nbr_51',
       'store_nbr

## 2.4: Split Data Back into Train and Test Sets

In [38]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Step 5: Split Data Based on Time Series Order

# Separate train and test datasets
train_df = df[df['dataset'] == 'train'].drop(columns=['dataset'], errors='ignore')
test_df = df[df['dataset'] == 'test'].drop(columns=['dataset'], errors='ignore')


# Drop unnecessary columns from both datasets
train_df = train_df.drop(columns=['transactions', 'id'], errors='ignore')
test_df = test_df.drop(columns=['transactions'], errors='ignore')

# Sort training data by date to preserve time series order
train_df = train_df.sort_values(by='date')

# Define the split point for time series validation
split_ratio = 0.99  # Adjust as needed
split_index = int(len(train_df) * split_ratio)

# Create training and validation sets
X_train = train_df.iloc[:split_index].drop(columns=['sales'])
y_train = train_df.iloc[:split_index]['sales']
X_val = train_df.iloc[split_index:].drop(columns=['sales'])
y_val = train_df.iloc[split_index:]['sales']

# Log transform the target variables to stabilize variance
y_train = np.log1p(y_train)  # Log transform training target
y_val = np.log1p(y_val)      # Log transform validation target

# Drop the 'date' column after feature extraction
X_train = X_train.drop(columns=['date'], errors='ignore')
X_val = X_val.drop(columns=['date'], errors='ignore')

X_test = test_df.drop(columns=['sales'], errors='ignore')

In [39]:
X_val

,onpromotion,dcoilwtico,is_holiday,store_nbr_2,store_nbr_3,store_nbr_4,store_nbr_5,store_nbr_6,store_nbr_7,store_nbr_8,...,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD,month_encoded,day_encoded,year_encoded,day_of_week_encoded
3023829,0,49.72,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,376.414091,345.823234,481.166458,434.785811
3023817,0,49.72,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,376.414091,345.823234,481.166458,434.785811
3023806,1,49.72,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,376.414091,345.823234,481.166458,434.785811
3023807,0,49.72,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,376.414091,345.823234,481.166458,434.785811
3023808,12,49.72,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,376.414091,345.823234,481.166458,434.785811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054345,148,47.57,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,336.992535,348.736707,481.166458,319.920782
3054346,8,47.57,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,336.992535,348.736707,481.166458,319.920782
3054347,0,47.57,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,336.992535,348.736707,481.166458,319.920782
3054322,25,47.57,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,336.992535,348.736707,481.166458,319.920782


# 3. Model Selection and Implementation

## CatBoost

In [40]:
# Define train and validation pools
from catboost import CatBoostRegressor, Pool

# Assuming train_pool and val_pool are already defined
train_pool = Pool(X_train, y_train)
val_pool = Pool(X_val, y_val)

# Define the CatBoost model with initial parameters
catboost_model = CatBoostRegressor(
    iterations=10000,          # Total iterations to train
    learning_rate=0.1,        # Learning rate
    depth=8,                  # Tree depth
    loss_function='RMSE',     # Loss function
    eval_metric='RMSE',       # Evaluation metric
    random_seed=42,           # Random seed for reproducibility
    verbose=100,               # Verbose output for every 50 iterations
    early_stopping_rounds=50  # Early stopping if no improvement for 50 rounds
)

# Training loop
train_loop_count = 100  # Number of training stages
init_model = None  # Start without an initial model

for i in range(train_loop_count):
    print(f"\nTraining Stage {i + 1}/{train_loop_count}\n")
    catboost_model.fit(
        train_pool,
        eval_set=val_pool,
        early_stopping_rounds=50,
        verbose=50,
        use_best_model=True,
        init_model=init_model  # Use the model from the previous stage
    )
    # Update the initial model for the next training stage
    init_model = catboost_model

# Final model is stored in catboost_model


Training Stage 1/100

0:	learn: 2.5592675	test: 2.4468546	best: 2.4468546 (0)	total: 301ms	remaining: 50m 7s
50:	learn: 1.2962385	test: 1.0346200	best: 1.0346200 (50)	total: 10.2s	remaining: 33m 11s
100:	learn: 1.0727111	test: 0.8925951	best: 0.8925951 (100)	total: 19.4s	remaining: 31m 39s
150:	learn: 0.9675240	test: 0.8060438	best: 0.8060438 (150)	total: 28.4s	remaining: 30m 55s
200:	learn: 0.9001200	test: 0.7506687	best: 0.7506687 (200)	total: 38s	remaining: 30m 50s
250:	learn: 0.8474015	test: 0.7144918	best: 0.7144918 (250)	total: 47s	remaining: 30m 25s
300:	learn: 0.8100425	test: 0.6889586	best: 0.6889586 (300)	total: 56s	remaining: 30m 3s
350:	learn: 0.7757096	test: 0.6689362	best: 0.6689362 (350)	total: 1m 5s	remaining: 29m 49s
400:	learn: 0.7515872	test: 0.6504629	best: 0.6504629 (400)	total: 1m 14s	remaining: 29m 47s
450:	learn: 0.7308678	test: 0.6346199	best: 0.6346199 (450)	total: 1m 23s	remaining: 29m 35s
500:	learn: 0.7107595	test: 0.6235811	best: 0.6235811 (500)	total: 1m

In [41]:
august_pool = Pool(X_val, y_val)

In [42]:
for i in range(1):
    catboost_model.fit(august_pool, verbose=50, init_model = catboost_model)

0:	learn: 0.4636497	total: 4.05ms	remaining: 40.5s
50:	learn: 0.4147613	total: 174ms	remaining: 34s
100:	learn: 0.4022951	total: 345ms	remaining: 33.8s
150:	learn: 0.3909944	total: 514ms	remaining: 33.5s
200:	learn: 0.3827195	total: 688ms	remaining: 33.6s
250:	learn: 0.3760679	total: 858ms	remaining: 33.3s
300:	learn: 0.3704810	total: 1.03s	remaining: 33.2s
350:	learn: 0.3654578	total: 1.2s	remaining: 33s
400:	learn: 0.3609986	total: 1.37s	remaining: 32.8s
450:	learn: 0.3569799	total: 1.54s	remaining: 32.7s
500:	learn: 0.3532494	total: 1.73s	remaining: 32.9s
550:	learn: 0.3498452	total: 1.9s	remaining: 32.6s
600:	learn: 0.3465577	total: 2.07s	remaining: 32.4s
650:	learn: 0.3436464	total: 2.24s	remaining: 32.2s
700:	learn: 0.3408559	total: 2.42s	remaining: 32.1s
750:	learn: 0.3380839	total: 2.59s	remaining: 31.9s
800:	learn: 0.3354737	total: 2.76s	remaining: 31.7s
850:	learn: 0.3329439	total: 2.93s	remaining: 31.5s
900:	learn: 0.3306899	total: 3.11s	remaining: 31.4s
950:	learn: 0.328547

# Submition

In [43]:
# Preprocess test data
test_features = test_df.drop(columns=['id', 'date'], errors='ignore')  # Drop unnecessary columns
test_features = test_features.reindex(columns=X_train.columns, fill_value=0)  # Align columns with training data

In [44]:
# Predict using the trained CatBoost model
test_df['sales'] = catboost_model.predict(test_features)

In [45]:
# Reverse log transformation if applied during training
test_df['sales'] = np.expm1(test_df['sales'])  # Use np.expm1 if np.log1p was used during training

# Clip predictions to avoid negative values (optional)
test_df['sales'] = test_df['sales'].clip(lower=0)

In [46]:
# Create submission file
submission = test_df[['id', 'sales']]  # Include 'id' and the predicted target column
submission.to_csv('submission.csv', index=False)

print("Submission file created: submission.csv")

Submission file created: submission.csv
